### **Tensorflow8강 Constructing CNN2**

**fully connected layer**
![ex8_1](./image/8_1.png)

**loss**
![ex8_2](./image/8_2.png)

**AdamOptimizer**  
mini-batch로 gradient decent를 시키면 minimum과 가까워지긴 하지만 진동을 발생한다.
관성과 비슷하게 momentum을 주게되면(수치해석적으로 무게를 주게되면) 방향을 쉽게 바꾸지 못한다.
![ex8_3](./image/8_3.png)

**drop out**  
overfitting을 막아주고 balancing을 잡아주고, 연산량도 줄여준다.

In [1]:
import tensorflow as tf
import os
from PIL import Image  # image 출력을 위한 library

In [2]:
image_dir = os.getcwd() + "/test_image/jooseok_gray.png"
label_dir = os.getcwd() + "/test_image/label.csv"

image_list = [image_dir]
label_list = [label_dir]

image_queue = tf.train.string_input_producer(image_list)
label_queue = tf.train.string_input_producer(label_list)

image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()

image_key, image_value = image_reader.read(image_queue)
label_key, label_value = label_reader.read(label_queue)

image = tf.image.decode_png(image_value)
label = tf.decode_csv(label_value, record_defaults=[[0]])

x = tf.cast(image, tf.float32)
y_ = tf.cast(label, tf.float32)

y_ = tf.reshape(y_, [-1, 1])

In [3]:
IMAGE_WIDTH =50
IMAGE_HEIGHT = 50
#HIDDEN1_SIZE = 32
#HIDDEN2_SIZE = 64
#CLASSES = 10

In [4]:
hidden1_w = tf.Variable(tf.truncated_normal([5, 5, 1, 32]))  # width, height, depth, batch
hidden1_b = tf.Variable(tf.zeros([32]))

hidden2_w = tf.Variable(tf.truncated_normal([5, 5, 32, 64]))
hidden2_b = tf.Variable(tf.zeros([64]))


fc_w = tf.Variable(tf.truncated_normal([IMAGE_WIDTH * IMAGE_HEIGHT * 64, 10]))  # CLASS는 임의의 숫자 
fc_b = tf.Variable(tf.zeros([10]))

out_w = tf.Variable(tf.truncated_normal([10, 1]))
out_b = tf.Variable(tf.zeros([1]))

In [5]:
# convolution
x_image = tf.reshape(x, [-1, IMAGE_WIDTH, IMAGE_HEIGHT, 1])  # reshape를 해주어야 dimension오류가 없다.

h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, hidden1_w, strides=[1, 1, 1, 1], padding="SAME") + hidden1_b)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, hidden2_w, strides=[1, 1, 1, 1], padding="SAME") + hidden2_b)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="SAME")

In [6]:
# fully connected layer
h_flat = tf.reshape(h_pool2, [-1, IMAGE_WIDTH * IMAGE_HEIGHT * 64])
h_fc1 = tf.nn.softmax(tf.matmul(h_flat, fc_w) + fc_b)

drop_fc = tf.nn.dropout(h_fc1, 0.7)
pred = tf.matmul(drop_fc, out_w) + out_b

In [7]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y_))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [8]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))  # 1: reduce할 때, 어느 방향으로 할꺼
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
 with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    sess.run(tf.global_variables_initializer())
    
    for i in range(10):
        sess.run(train)
        _loss, _accuracy = sess.run([loss, accuracy])
        print("=========================")
        print("loss: ", _loss)
        print("accuracy: ", _accuracy)
    
    coord.request_stop()
    coord.join(threads)

loss:  21.918861
accuracy:  1.0
loss:  0.69004714
accuracy:  1.0
loss:  0.68849725
accuracy:  1.0
loss:  21.907354
accuracy:  1.0
loss:  21.903517
accuracy:  1.0
loss:  21.899994
accuracy:  1.0
loss:  21.896397
accuracy:  1.0
loss:  0.6807501
accuracy:  1.0
loss:  21.889061
accuracy:  1.0
loss:  0.677652
accuracy:  1.0
